In [4]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np




In [9]:
price_response_df = pd.read_csv('/Users/youziya/Library/CloudStorage/OneDrive-YorkUniversity/OMIS 6000 Models and Applications in Operational Reseach/Assignment 2/price_response.csv')

a)

In [6]:


# Define the coefficients for the price response functions
a1, b1 = 35234.545786, 45.896450
a2, b2 = 37790.240832, 8.227794

# Define the objective function to be maximized (Note: minimize function is used, so we take the negative of the objective)
def objective(prices):
    p1, p2 = prices
    return -((p1 * (a1 - b1 * p1)) + (p2 * (a2 - b2 * p2)))

# Define the constraints
constraints = [
    {'type': 'ineq', 'fun': lambda prices: prices[1] - prices[0]},  # p2 - p1 >= 0 (Advanced price higher than Basic)
    {'type': 'ineq', 'fun': lambda prices: prices[0]},  # p1 >= 0
    {'type': 'ineq', 'fun': lambda prices: prices[1]}   # p2 >= 0
]

# Initial guess for the prices
initial_guess = [10, 20]

# Perform the optimization
result = minimize(objective, initial_guess, constraints=constraints)

result.x  # Optimal prices for Product 1 and Product 2

array([ 383.84823389, 2296.49742446])

b)

In [10]:
from scipy.optimize import Bounds
from gurobipy import Model, GRB, quicksum

# Initialize model
model = Model("Optimal Pricing Strategy")

# Define variables (prices) for all nine products based on dataset structure
prices = model.addVars(9, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Price")

# Coefficients from the dataset
intercepts = price_response_df['Intercept'].values
sensitivities = price_response_df['Sensitivity'].abs().values  # Take absolute value to ensure positive sensitivity

# Define the objective function
objective = quicksum((prices[i] * (intercepts[i] - sensitivities[i] * prices[i])) for i in range(9))

# Set objective
model.setObjective(objective, GRB.MAXIMIZE)

# Define constraints based on cross-elasticity (not explicitly given, but assuming need for non-negativity and potential capacity constraints)
# Here, we are primarily focused on the pricing strategy maximization, additional constraints related to cross-elasticity and capacity are not explicitly defined

# Optimize model
model.optimize()

# Extract optimal prices
optimal_prices = [prices[i].X for i in range(9)]

optimal_prices


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 9 columns and 0 nonzeros
Model fingerprint: 0xfa548505
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 9 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.04 seconds (0.00 work units)
Optimal objective 7.38368672e+08


[383.84827160837136,
 2296.498918132054,
 2351.877667019544,
 2050.2987944234346,
 4160.707856083366,
 6813.656504083377,
 8138.453042277662,
 4498.417684502101,
 8558.942360734147]

In [11]:
# Given that we cannot use Gurobi for the quadratic programming step, let's implement a simplified version
# of the projected gradient descent algorithm for the optimization problem.

def gradient_descent_step(prices, intercepts, sensitivities, step_size=0.001):
    """
    Perform one step of gradient descent and project onto the feasible set.
    
    prices: Current prices
    intercepts: Intercept terms of the price response functions
    sensitivities: Sensitivity terms of the price response functions
    step_size: Step size for the gradient descent
    """
    # Calculate gradient of objective function with respect to prices
    gradients = -2 * sensitivities * prices + intercepts
    
    # Update prices using gradient
    new_prices = prices + step_size * gradients
    
    # Projection step: Ensure non-negativity and price ordering if necessary
    # For simplicity, we'll just enforce non-negativity here as the specific projection onto a set
    # defined by linear constraints would require solving a quadratic program.
    projected_prices = np.maximum(new_prices, 0)
    
    return projected_prices

# Initialize all prices to zero
initial_prices = np.zeros(9)

# Set step size and stopping criterion
step_size = 0.001
tolerance = 1e-6

# Perform gradient descent until stopping criterion is met
current_prices = initial_prices
iteration = 0
while True:
    next_prices = gradient_descent_step(current_prices, intercepts, sensitivities, step_size)
    # Check stopping criterion (norm of gradient less than tolerance)
    if np.linalg.norm(next_prices - current_prices) < tolerance:
        break
    current_prices = next_prices
    iteration += 1
    if iteration % 100 == 0:  # Output every 100 iterations to monitor progress
        print(f"Iteration {iteration}: Prices = {current_prices}")

current_prices


Iteration 100: Prices = [ 383.82299907 1859.51670611 1841.85274264 1719.14309511 2451.26608747
 2791.8561638  3130.0013826  2483.84877279 3157.88892293]
Iteration 200: Prices = [ 383.84826994 2213.34910599 2241.27437028 1996.81190876 3458.37756486
 4439.76548574 5056.22256223 3596.21424661 5150.64995631]
Iteration 300: Prices = [ 383.84827161 2280.67701264 2327.8923897  2041.65981601 3872.15293305
 5412.45353771 6241.63035581 4094.37539108 6408.1661746 ]
Iteration 400: Prices = [ 383.84827161 2293.48829566 2346.67625309 2048.90346271 4042.15402942
 5986.5883119  6971.13725354 4317.47160514 7201.71191981]
Iteration 500: Prices = [ 383.84827161 2295.92605112 2350.74969562 2050.07342627 4111.99959053
 6325.47469821 7420.08007028 4417.38289144 7702.47273752]
Iteration 600: Prices = [ 383.84827161 2296.3899119  2351.63305671 2050.2623939  4140.69589021
 6525.50435673 7696.36210863 4462.12710716 8018.47392341]
Iteration 700: Prices = [ 383.84827161 2296.47817622 2351.82462117 2050.29291516 4

array([ 383.84827161, 2296.49891813, 2351.87766702, 2050.29879442,
       4160.70785608, 6813.65649147, 8138.45296829, 4498.4176845 ,
       8558.94215813])

In [12]:
# Correcting the oversight by defining intercepts and sensitivities from the dataset for all products
intercepts = price_response_df['Intercept'].values
sensitivities = price_response_df['Sensitivity'].abs().values  # Taking absolute value to ensure positive sensitivity

# Reinitialize prices to zero
initial_prices = np.zeros(len(intercepts))

# Re-run the simplified projected gradient descent with correct intercepts and sensitivities
current_prices = initial_prices
iteration = 0
while True:
    next_prices = gradient_descent_step(current_prices, intercepts, sensitivities, step_size)
    # Check stopping criterion (norm of the difference less than tolerance)
    if np.linalg.norm(next_prices - current_prices) < tolerance:
        break
    current_prices = next_prices
    iteration += 1

# Since the loop doesn't contain print statements this time, we directly output the final prices
current_prices


array([ 383.84827161, 2296.49891813, 2351.87766702, 2050.29879442,
       4160.70785608, 6813.65649147, 8138.45296829, 4498.4176845 ,
       8558.94215813])

c)

In [25]:
from gurobipy import Model, GRB

# Create a new Gurobi model
model = Model("OptimalPricingStrategy")

# Assuming hypothetical linear price-response functions for simplicity
# a_ij and b_ij could be specific to each product version in each product line
# For example purposes, these are generic coefficients
a = {('Basic', 'EvoTech'): 1000, ('Advanced', 'EvoTech'): 1500, ('Premium', 'EvoTech'): 2000,
     ('Basic', 'InfiniteEdge'): 1100, ('Advanced', 'InfiniteEdge'): 1600, ('Premium', 'InfiniteEdge'): 2100,
     ('Basic', 'FusionBook'): 1200, ('Advanced', 'FusionBook'): 1700, ('Premium', 'FusionBook'): 2200}
b = {version: 10 for version in a}  # Assuming a uniform sensitivity for simplicity

# Decision variables for prices
prices = model.addVars(a.keys(), name="P", lb=0, vtype=GRB.CONTINUOUS)

# Objective: Maximize total revenue
model.setObjective(sum((prices[version] * (a[version] - b[version] * prices[version])) for version in a), GRB.MAXIMIZE)

# Constraints: Prices within each product line increase from Basic to Advanced to Premium
model.addConstr(prices[('Basic', 'EvoTech')] <= prices[('Advanced', 'EvoTech')], "EvoTech_Basic_Advanced")
model.addConstr(prices[('Advanced', 'EvoTech')] <= prices[('Premium', 'EvoTech')], "EvoTech_Advanced_Premium")

model.addConstr(prices[('Basic', 'InfiniteEdge')] <= prices[('Advanced', 'InfiniteEdge')], "InfiniteEdge_Basic_Advanced")
model.addConstr(prices[('Advanced', 'InfiniteEdge')] <= prices[('Premium', 'InfiniteEdge')], "InfiniteEdge_Advanced_Premium")

model.addConstr(prices[('Basic', 'FusionBook')] <= prices[('Advanced', 'FusionBook')], "FusionBook_Basic_Advanced")
model.addConstr(prices[('Advanced', 'FusionBook')] <= prices[('Premium', 'FusionBook')], "FusionBook_Advanced_Premium")

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for version in a:
        var = prices[version]  # Directly access the variable from the dictionary
        print(f"{version[1]} {version[0]} Price: ${var.X:.2f}")
    print(f"Optimal Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x91060efe
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+03]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.31200000e+08  3.60000000e+08  0.00e+00 0.00e+00  9.90e+0

d)

In [23]:
from gurobipy import Model, GRB

# Example data structures for intercepts and sensitivities
# Adjust with your actual data
intercepts = {(i, j): 10000 for i in range(3) for j in range(3)}  # Placeholder values
sensitivities = {(i, j): 50 for i in range(3) for j in range(3)}  # Placeholder values

# Initialize the model
model = Model("TechEssentials Pricing Optimization")

# Define decision variables for prices
prices = model.addVars(3, 3, lb=0, name="P", vtype=GRB.CONTINUOUS)

# Set the objective: Maximize total revenue
model.setObjective(sum(prices[i, j] * (intercepts[i, j] - sensitivities[i, j] * prices[i, j])
                       for i in range(3) for j in range(3)), GRB.MAXIMIZE)

# Constraint 1: Prices increase within each product line (Basic < Advanced < Premium)
for i in range(3):
    model.addConstr(prices[i, 0] <= prices[i, 1], f"Line_{i}_Basic_Advanced")
    model.addConstr(prices[i, 1] <= prices[i, 2], f"Line_{i}_Advanced_Premium")

# Constraint 2: Prices increase across product lines for the same version
# EvoTech Pro < InfiniteEdge Notebook < FusionBook Elite for each version
# Additional constraints for part (d) ensuring inter-line price increase for the same versions
for j in range(3):  # For each version (Basic, Advanced, Premium)
    model.addConstr(prices[(0, j)] <= prices[(1, j)], "InterLine_Basic_EvoTech_InfiniteEdge")
    model.addConstr(prices[(1, j)] <= prices[(2, j)], "InterLine_Advanced_InfiniteEdge_FusionBook")


# Solve the model
model.optimize()

# Output the optimal prices and the anticipated optimal revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices for Each Product Version in Each Product Line:")
    for i in range(3):
        for j in range(3):
            print(f"Product Line {i+1}, Version {j+1}: ${prices[i, j].X:.2f}")
    print(f"\nAnticipated Optimal Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x0fe8be80
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 1e+04]
  QObjective range [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.62000000e+09  1.80000000e+09  0.00e+00 0.00e+00  9.87e

In [26]:
from gurobipy import Model, GRB

# Initialize the Gurobi model
model = Model("OptimalPricingStrategyEnhanced")

# Define decision variables for prices, considering three product lines and three versions each
# Using a simplified structure for indices: (product_line, version)
# Product lines: 0 = EvoTech Pro, 1 = InfiniteEdge Notebook, 2 = FusionBook Elite
# Versions: 0 = Basic, 1 = Advanced, 2 = Premium
prices = model.addVars([(line, version) for line in range(3) for version in range(3)], name="price", lb=0, vtype=GRB.CONTINUOUS)

# Example intercepts and sensitivities for the price-demand function, to be customized as per actual data
intercepts = {(line, version): 1000 - 10*line + 100*version for line in range(3) for version in range(3)}
sensitivities = {(line, version): 2 + 0.5*version for line in range(3) for version in range(3)}

# Set the objective: Maximize total revenue
model.setObjective(
    sum(prices[(line, version)] * (intercepts[(line, version)] - sensitivities[(line, version)] * prices[(line, version)])
        for line in range(3) for version in range(3)), GRB.MAXIMIZE)

# Add constraints for increasing prices within each product line
for line in range(3):
    for version in range(2):  # Only need to compare Basic to Advanced, and Advanced to Premium
        model.addConstr(prices[(line, version)] <= prices[(line, version+1)], f"WithinLine_{line}_{version}")

# Add constraints for increasing prices across product lines for the same version
for version in range(3):
    model.addConstr(prices[(0, version)] <= prices[(1, version)], f"AcrossLines_0_{version}")
    model.addConstr(prices[(1, version)] <= prices[(2, version)], f"AcrossLines_1_{version}")

# Solve the optimization problem
model.optimize()

# Output the optimal prices and anticipated revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices for Each Version in Each Product Line:")
    for line in range(3):
        for version in range(3):
            print(f"Product Line {line+1}, Version {version+1}: ${prices[(line, version)].X:.2f}")
    print(f"Optimal Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x86c15e53
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+03]
  QObjective range [4e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.20760000e+08  3.60000000e+08  0.00e+00 0.00e+00  9.77e

e)In practice, the most sensible prices to implement would ideally follow the logic of (d), where there's a clear differentiation in pricing both within each product line and across product lines for the same versions, reflecting both product value and strategic market positioning. However, it's critical to ensure that the optimization model correctly implements these constraints and that the output accurately reflects this logic. The reported identical prices for all versions in the FusionBook line under (d) suggest a need to revisit the model or output interpretation to ensure it aligns with practical business strategies and product differentiation goals.

f) the consumer behaviour might change nd hard for us to predict. there are also external factors and economic condition.